In [1]:
# import all required labraries

import numpy as np
import pandas as pd
import tensorflow as tf
import os
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm

# save and run

# now define path to dataset
path="asl_alphabet_train/asl_alphabet_train"
files=os.listdir(path)
# list of files in path
# sort path from A-Y
files.sort()

# print to see list
print(files)


# create list of image and label

image_array=[]
label_array=[]
# loop through each file in files

for i in tqdm(range(len(files))):
	# list of image in each folder
	sub_file=os.listdir(path+"/"+files[i])
	# let's check length of each folder
	#	print(len(sub_file))

	# loop through each sub folder
	for j in range(len(sub_file)):

		# path of each image
		#Example:imagepro/A/image_name1.jpg

		file_path=path+"/"+files[i]+"/"+sub_file[j]
		# read each image

		image=cv2.imread(file_path)

		# resize image by 96x96
		image=cv2.resize(image,(96,96))
		# convert BGR image to RGB image
		image=cv2.cvtColor(image,cv2.COLOR_BGR2RGB)

		# add this image at image_array
		image_array.append(image)

		# add label to label_array
		# i is number from 0 to len(files)-1
		# so we can use it as label
		label_array.append(i)

# save and run to see if it is working or not
# before that apply tqdm to for loop
# it is working with no errors

# convert list to array

image_array=np.array(image_array)
label_array=np.array(label_array,dtype="float")

# split the dataset into test and train

from sklearn.model_selection import train_test_split
# output									   train image   label      spliting size
X_train,X_test,Y_train,Y_test=train_test_split(image_array,label_array,test_size=0.15)

del image_array,label_array

# to free memory 
import gc
gc.collect()


# X_train will have 85% of images 
# X_test will have 15% of images


# Create a model

from keras import layers,callbacks,utils,applications,optimizers
from keras.models import Sequential, Model, load_model

model=Sequential()
# add pretrained models to Sequential model
# I will use EfficientNetB0 pretrained model. You can try different model.
pretrained_model=tf.keras.applications.EfficientNetB0(input_shape=(96,96,3),include_top=False)
model.add(pretrained_model)

# add Pooling to model
model.add(layers.GlobalAveragePooling2D())

# add dropout to model
# We add dropout to increase accuracy by reduce overfitting
model.add(layers.Dropout(0.3))
# finally we will addd dense layer as an output
model.add(layers.Dense(1))
# For some tensorflow version we required to build model
model.build(input_shape=(None,96,96,3))


# to see model summary
model.summary()


# save and run to see model summary 
# make sure your pc is connected to internet to download pretrained weight
# It will take some time
# Everything till now works
# I am using GPU to train model so it will take 20-30 min.
# If you train model on CPU it will take some time


# compile model
# you can use different optimizer and loss function to increase accuracy
model.compile(optimizer="adam",loss="mae",metrics=["mae"])

# create a checkpoint to save best accuracy model
ckp_path="trained_model/model"
model_checkpoint=tf.keras.callbacks.ModelCheckpoint(
									filepath=ckp_path,
									monitor="val_mae",
									mode="auto",
									save_best_only=True,
									save_weights_only=True
									)
# monitor: monitor validation mae loss to save model
# mode: Use to save model when val_mae is minimum or maximum
# It has 3 option: "min","max","auto".
# for us you can select either "min" or "auto"
# When val_mae reduce model will be saved
# save_best_only: False -> It will save all model
# save_weights_only: Save only weight.


# create learning rate reducer to reduce lr when accuracy does not improve
# Correct 
reduce_lr=tf.keras.callbacks.ReduceLROnPlateau(
									factor=0.9,
									monitor="val_mae",
									mode="auto",
									cooldown=0,
									patience=5,
									verbose=1,
									min_lr=1e-6)

# factor: when it is reduce next lr will be 0.9 times of current
# next lr= 0.9* current lr

# patience=X
# reduce lr after X epoch when accuracy does not improve
# verbose : show it after every epoch

# min_lr : minimum learning rate 

# Start training model

Epochs=5
Batch_Size=32
# Select batch size according to your Graphic card 
#
#X_train,X_test,Y_train,Y_test
with tf.device('/device:GPU:0'):
	history=model.fit(
					X_train,
					Y_train,
					validation_data=(X_test,Y_test),
					batch_size=Batch_Size,
					epochs=Epochs,
					callbacks=[model_checkpoint,reduce_lr]
					)
# Before training you can delete image_array and label_array to increase ram memory 

# Some error correction
# This code will be in the description so you can cross check everything
# Save and run 
# Everything is working

# after the training is done load best model

model.load_weights(ckp_path)

# convert model to tensorflow lite model

converter=tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model=converter.convert()

# save model
with open("model.tflite","wb") as f:
	f.write(tflite_model)

# if you want to see prediction result on test dataset
prediction_val=model.predict(X_test,batch_size=32)

# print first 10 values
print(prediction_val)
# print first 10 values of Y_test
print(Y_test)

# Save and run this python file
# Before that I will show you
# loss: 0.4074 - mae: 0.4074 - val_loss: 0.3797 - val_mae: 0.3797
# we have mae and val_mae:
# mae: Is on X_train
# val_mae: X_test
# If val_mae is reducing that means your model is improving.

# I will show you the result after the training is over


['A', 'B', 'C', 'D', 'E']


100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:18<00:00,  3.67s/it]


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 96, 96, 3)         0         
                                                                 
 conv2d (Conv2D)             (None, 96, 96, 16)        448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 48, 48, 16)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 48, 48, 32)        4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 24, 24, 32)       0         
 2D)                                                             
                                                                 
 dropout (Dropout)           (None, 24, 24, 32)        0

INFO:tensorflow:Assets written to: C:\Users\DELVIN\AppData\Local\Temp\tmpg0tqxd_4\assets


INFO:tensorflow:Assets written to: C:\Users\DELVIN\AppData\Local\Temp\tmpg0tqxd_4\assets


12/12 [==============================] - 0s 22ms/step
[[1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.

In [4]:
# now define path to dataset
path="DownloadedSamples"
files=os.listdir(path)
# list of files in path
# sort path from A-Y
files.sort()

# print to see list
print(files)


# create list of image and label

image_array=[]
label_array=[]
# loop through each file in files

#for i in tqdm(range(len(files))):
	# list of image in each folder
	#sub_file=os.listdir(path+"/"+files[i])
	# let's check length of each folder
	#	print(len(sub_file))

i=0
# loop through each sub folder
for j in range(len(files)):

    # path of each image
    #Example:imagepro/A/image_name1.jpg

    file_path=path+"/"+files[j]
    # read each image

    image=cv2.imread(file_path)

    # resize image by 96x96
    image=cv2.resize(image,(96,96))
    # convert BGR image to RGB image
    image=cv2.cvtColor(image,cv2.COLOR_BGR2RGB)

    # add this image at image_array
    image_array.append(image)

    # add label to label_array
    # i is number from 0 to len(files)-1
    # so we can use it as label
    label_array.append(i)
    i=i+1

# save and run to see if it is working or not
# before that apply tqdm to for loop
# it is working with no errors

# convert list to array

image_array=np.array(image_array)
label_array=np.array(label_array,dtype="float")

# if you want to see prediction result on test dataset
prediction_val=model.predict(image_array,batch_size=32)

# print first 10 values
print(prediction_val)
# print first 10 values of Y_test
print(label_array)

import string
leter=()
leter=list(string.ascii_uppercase)
print(leter)

j=0
for i in range(len(files)):
    
    #print(prediction_val[i], " Act=", leter[i], " Pred=", end=" ")
    print(prediction_val[i], " Pred=", end=" ")
    
    if prediction_val[i]>=-0.5 and prediction_val[i]<0.5:
        print("= A")
    elif prediction_val[i]>=-0.5 and prediction_val[i]<1.5:
        print("= B")
    elif prediction_val[i]>=-1.5 and prediction_val[i]<2.5:
        print("= C")
    elif prediction_val[i]>=-2.5 and prediction_val[i]<3.5:
        print("= D")
    elif prediction_val[i]>=-3.5 and prediction_val[i]<4.5:
        print("= E")
    elif prediction_val[i]>=-4.5 and prediction_val[i]<5.5:
        print("= F")
    elif prediction_val[i]>=-5.5 and prediction_val[i]<6.5:
        print("= G")
    elif prediction_val[i]>=-6.5 and prediction_val[i]<7.5:
        print("= H")
    elif prediction_val[i]>=-7.5 and prediction_val[i]<8.5:
        print("= I")
    elif prediction_val[i]>=-8.5 and prediction_val[i]<9.5:
        print("= J")
        
    elif prediction_val[i]>=-9.5 and prediction_val[i]<10.5:
        print("= K")
    elif prediction_val[i]>=-10.5 and prediction_val[i]<11.5:
        print("= L")
    elif prediction_val[i]>=-11.5 and prediction_val[i]<12.5:
        print("= M")
    elif prediction_val[i]>=-12.5 and prediction_val[i]<13.5:
        print("= N")
    elif prediction_val[i]>=-13.5 and prediction_val[i]<14.5:
        print("= O")
    elif prediction_val[i]>=-14.5 and prediction_val[i]<15.5:
        print("= P")
    elif prediction_val[i]>=-15.5 and prediction_val[i]<16.5:
        print("= Q")
    elif prediction_val[i]>=-16.5 and prediction_val[i]<17.5:
        print("= R")
    elif prediction_val[i]>=-17.5 and prediction_val[i]<18.5:
        print("= S")
    elif prediction_val[i]>=-18.5 and prediction_val[i]<19.5:
        print("= T")
    
    elif prediction_val[i]>=-19.5 and prediction_val[i]<20.5:
        print("= U")
    elif prediction_val[i]>=-20.5 and prediction_val[i]<21.5:
        print("= V")
    elif prediction_val[i]>=-21.5 and prediction_val[i]<22.5:
        print("= W")
    elif prediction_val[i]>=-22.5 and prediction_val[i]<23.5:
        print("= X")
    elif prediction_val[i]>=-23.5 and prediction_val[i]<24.5:
        print("= Y")
    elif prediction_val[i]>=-24.5 and prediction_val[i]<25.5:
        print("= Z")
    elif prediction_val[i]>=-25.5 and prediction_val[i]<26.5:
        print("= DEL")
    elif prediction_val[i]>=-26.5 and prediction_val[i]<27.5:
        print("= NOTHING")
    elif prediction_val[i]>=-27.5 and prediction_val[i]<28.5:
        print("= SPACE")
    else:
        print("= Dont Know!")


['E_test.jpg']
1/1 [==============================] - 0s 16ms/step
[[1.]]
[0.]
['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']
[1.]  Pred= = B


In [8]:
import tensorflow as tf

#load the model to convert it.
model = load_model('model2.h5')

# Convert the model. from hd5 model to tflite. U sure about that?its directly from model
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

# Save the model.
with open('model_h5.tflite', 'wb') as f:
  f.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\DELVIN\AppData\Local\Temp\tmppkl7olue\assets


INFO:tensorflow:Assets written to: C:\Users\DELVIN\AppData\Local\Temp\tmppkl7olue\assets
